In [11]:
import pickle
import pandas as pd
import nltk
import re

In [28]:
import pandas as pd
import re
import unicodedata

def limpiar_texto(texto):
    
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    texto_limpio = re.sub(r'[^a-zA-Z0-9\s]', '', texto_normalizado)
    return texto_limpio.lower()



In [29]:
puestos = pickle.load(open('static/data/puestos.pickle', 'rb'))
puestos["PUESTO"] = puestos["PUESTO"].apply(limpiar_texto)

In [31]:
### Tokenización
puestos["tokens"] = puestos["PUESTO"].apply(nltk.word_tokenize)

In [ ]:
### Stop words

stopwords = nltk.corpus.stopwords.words('spanish')
puestos["tokens_sin_stopwords"] = puestos["tokens"].apply(lambda x: [word for word in x if word not in stopwords])

In [34]:
### Lematización

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

puestos["tokens_lematizados"] = puestos["tokens_sin_stopwords"].apply(lambda x: [wordnet_lemmatizer.lemmatize(word, pos = "v") for word in x])

In [ ]:
## Unir
puestos["texto_final"] = puestos["tokens_lematizados"].apply(lambda x: " ".join(x))

In [ ]:
### Vectorización

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorizer.fit(puestos["texto_final"])

In [ ]:
pickle.dump(vectorizer, open('static/data/nuestro_vectorizer.pickle', 'wb'))

In [ ]:
pickle.load(open('static/data/nuestro_vectorizer.pickle', 'rb'))

In [ ]:
import docx
doc = docx.Document("/Users/aladelca/Library/CloudStorage/GoogleDrive-alarcon.adrianc@gmail.com/My Drive/portafolio/cv_matcher/lab2/documents/Carlos Adrián Alarcón Delgado alarcon_prueba.docx")    
full_text = []
for para in doc.paragraphs:
    full_text.append(para.text)

full_text = " ".join(full_text)
texto_limpio = limpiar_texto(full_text)
tokens = nltk.word_tokenize(texto_limpio)
tokens_sin_stopwords = [word for word in tokens if word not in stopwords]
tokens_lematizados = [wordnet_lemmatizer.lemmatize(word, pos = "v") for word in tokens_sin_stopwords]
texto_final = " ".join(tokens_lematizados)
texto_final

In [55]:
puestos_vectorizados = vectorizer.transform(puestos["texto_final"]) 
documento_vectorizado = vectorizer.transform([texto_final])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similitudes = cosine_similarity(documento_vectorizado, puestos_vectorizados)
indices = pd.Series(similitudes[0]).sort_values(ascending = False).head(5).index

In [ ]:
puestos.iloc[indices, 0]